## Overall plan of approach
<br>
<b>
<li>1. Collecting data from xml annotations</li>
<li>2. Creating files for train and validation</li>
<li>3. Train using Faster RCNN model</li>
<li>4. Test and calculate MAP</li>
<br></b>

`In this notebook we will cover point 1 and 2`

In [1]:
#import packages
import xml.etree.ElementTree as ET
import pandas as pd
import xmltodict
import json
import os
from shutil import copy
import numpy as np
import cv2
import pickle

In [2]:
#read train txt and parse xml and add all information to list.
# here if there are 3 objects in a image then there will be 3 records. with same imagename but different object
train_txt_file = open('train.txt','r')
all_train_list=[]
for i in train_txt_file.readlines():
    if(i.strip()[-2:]=='_r'):
        pass
    else:
        with open('Annotations/'+i.strip()+'.xml') as fd:
            doc = xmltodict.parse(fd.read())
            json1 = json.dumps(doc)
            dictionary = json.loads(json1)
            if(str(type(dictionary['annotation']['object'])) =="<class 'list'>"):
                for i in range(len(dictionary['annotation']['object'])):
                    temp_list=[dictionary['annotation']['filename'],dictionary['annotation']['folder'],
                               dictionary['annotation']['size']['width'],dictionary['annotation']['size']['height'],
                               dictionary['annotation']['object'][i]['name'],
                               dictionary['annotation']['object'][i]['bndbox']['xmin'],
                               dictionary['annotation']['object'][i]['bndbox']['ymin'],
                               dictionary['annotation']['object'][i]['bndbox']['xmax'],
                               dictionary['annotation']['object'][i]['bndbox']['ymax']
                              ]
                    #print(temp_list)
                    all_train_list.append(temp_list)
            elif(str(type(dictionary['annotation']['object'])) =="<class 'dict'>"):
                temp_list=[dictionary['annotation']['filename'],dictionary['annotation']['folder'],
                               dictionary['annotation']['size']['width'],dictionary['annotation']['size']['height'],
                               dictionary['annotation']['object']['name'],dictionary['annotation']['object']['bndbox']['xmin'],
                               dictionary['annotation']['object']['bndbox']['ymin'],
                               dictionary['annotation']['object']['bndbox']['xmax'],
                               dictionary['annotation']['object']['bndbox']['ymax']]
                #print(temp_list)
                all_train_list.append(temp_list)
            else:
                print(type(dictionary['annotation']['object']))
                print('Take a look at xml file namend: ',i)
            

In [3]:
train_data_df = pd.DataFrame(all_train_list,columns=['filename','folder','width','height','class','xmin','ymin','xmax','ymax'])

### I have observed that the folder names mentioned in the xml so i have created separate logic to correct it `Cell 4-10`

<li>Moved all sub directories of given folders(frontFar, frontNear, rearNear, sideLeft, sideLeft, highquality_16k) to a newly created folder new</li>
<li>Based on folder names if all subdirectories separated them to different list</li>
<li>Sorted those newly created list by descending order of len</li>
<li>Based on given folder name in XML, seached corresponding list.</li>
<li>Created fully qualified path from root folder and tested if they are available or not.</li>
<li>Got perfect results where all files were available.</li>

In [4]:
list_of_folders_all=os.listdir('JPEGImages/all')

In [5]:
front_far=[]
front_near=[]
rear_near=[]
side_left=[]
side_right=[]
others=[]
part=[]

for i in list_of_folders_all:
    if 'frontFar' in i:
        front_far.append(i)
    elif 'frontNear' in i:
        front_near.append(i)
    elif 'rearNear' in i:
        rear_near.append(i)
    elif 'sideLeft' in i:
        side_left.append(i)
    elif 'sideRight' in i:
        side_right.append(i)
    elif 'part' in i:
        part.append(i)
    else:
        others.append(i)

In [6]:
front_far.sort(key=lambda x: len(x), reverse = True)
front_near.sort(key=lambda x: len(x), reverse = True)
rear_near.sort(key=lambda x: len(x), reverse = True)
side_left.sort(key=lambda x: len(x), reverse = True)
side_right.sort(key=lambda x: len(x), reverse = True)
part.sort(key=lambda x: len(x), reverse = True)
others.sort(key=lambda x: len(x), reverse = True)

In [7]:
def method_to_collect_actual_folder(s):
    if 'frontFar' in s:
        for i in front_far:
            if i in s:
                return i
    elif 'frontNear' in s:
        for i in front_near:
            if i in s:
                return i
    elif 'rearNear' in s:
        for i in rear_near:
            if i in s:
                return i
    elif 'sideLeft' in s:
        for i in side_left:
            if i in s:
                return i
    elif 'sideRight' in s:
        for i in side_right:
            if i in s:
                return i
    elif 'part' in s:
        for i in part:
            if i in s:
                return i
    else:
        for i in others:
            if i in s:
                return i

In [8]:
train_data_df['folder'] =  train_data_df.folder.apply(method_to_collect_actual_folder)

In [9]:
train_data_df['filly_qualified_path'] = 'JPEGImages/all/'+train_data_df.folder+'/'+train_data_df.filename

In [10]:
list_to_be_corrected =[]
count=0
count1=0
for i,j in enumerate(train_data_df['filly_qualified_path']):
    try:
        x = open(j)
        count1+=1
    except:
        count+=1
        list_to_be_corrected.append(j)
print(count)
print(count1)

0
328185


In [11]:
train_data_df.to_csv('train.csv')

`All files are available. Thats good news.`

### Lets repeat the same for validation

In [15]:
#read validation txt and parse xml and add all information to list.
# here if there are 3 objects in a image then there will be 3 records. with same imagename but different object
validation_txt_file = open('val.txt','r')
all_validation_list=[]
for i in validation_txt_file.readlines():
    if(i.strip()[-2:]=='_r'):
        pass
    else:
        with open('Annotations/'+i.strip()+'.xml') as fd:
            doc = xmltodict.parse(fd.read())
            json1 = json.dumps(doc)
            dictionary = json.loads(json1)
            if(str(type(dictionary['annotation']['object'])) =="<class 'list'>"):
                for i in range(len(dictionary['annotation']['object'])):
                    temp_list=[dictionary['annotation']['filename'],dictionary['annotation']['folder'],
                               dictionary['annotation']['size']['width'],dictionary['annotation']['size']['height'],
                               dictionary['annotation']['object'][i]['name'],
                               dictionary['annotation']['object'][i]['bndbox']['xmin'],
                               dictionary['annotation']['object'][i]['bndbox']['ymin'],
                               dictionary['annotation']['object'][i]['bndbox']['xmax'],
                               dictionary['annotation']['object'][i]['bndbox']['ymax']
                              ]
                    #print(temp_list)
                    all_validation_list.append(temp_list)
            elif(str(type(dictionary['annotation']['object'])) =="<class 'dict'>"):
                temp_list=[dictionary['annotation']['filename'],dictionary['annotation']['folder'],
                               dictionary['annotation']['size']['width'],dictionary['annotation']['size']['height'],
                               dictionary['annotation']['object']['name'],dictionary['annotation']['object']['bndbox']['xmin'],
                               dictionary['annotation']['object']['bndbox']['ymin'],
                               dictionary['annotation']['object']['bndbox']['xmax'],
                               dictionary['annotation']['object']['bndbox']['ymax']]
                #print(temp_list)
                all_validation_list.append(temp_list)
            else:
                print(type(dictionary['annotation']['object']))
                print('Take a look at xml file namend: ',i)
            

validation_data_df = pd.DataFrame(all_validation_list,columns=['filename','folder','width','height','class','xmin','ymin','xmax','ymax'])

In [16]:
validation_data_df['folder'] =  validation_data_df.folder.apply(method_to_collect_actual_folder)

In [17]:
validation_data_df['filly_qualified_path'] = 'JPEGImages/all/'+validation_data_df.folder+'/'+validation_data_df.filename

In [18]:
list_to_be_corrected =[]
count=0
count1=0
for i,j in enumerate(validation_data_df['filly_qualified_path']):
    try:
        x = open(j)
        count1+=1
    except:
        count+=1
        list_to_be_corrected.append(j)
print(count)
print(count1)

0
110440


In [19]:
validation_data_df.to_csv('validation.csv')

### Creating formatted data for RCNN model
<br>
<li>We will create data in format <b>['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class']</b></li>
<li>For each object there will be one entry. In other words if an image have multiple objects, then there will be multiple records for the same image. </li>
<br>

In [20]:
train = pd.read_csv('train.csv')
train.drop(columns=['Unnamed: 0'],inplace=True)
print(train.shape)
train.head()

(328185, 10)


,filename,folder,width,height,class,xmin,ymin,xmax,ymax,filly_qualified_path
0,0006780.jpg,BLR-2018-03-22_17-39-26_2_frontFar,1280,964,truck,586,494,638,551,JPEGImages/all/BLR-2018-03-22_17-39-26_2_front...
1,0006780.jpg,BLR-2018-03-22_17-39-26_2_frontFar,1280,964,vehicle fallback,470,483,595,613,JPEGImages/all/BLR-2018-03-22_17-39-26_2_front...
2,0006780.jpg,BLR-2018-03-22_17-39-26_2_frontFar,1280,964,car,638,471,726,537,JPEGImages/all/BLR-2018-03-22_17-39-26_2_front...
3,0006780.jpg,BLR-2018-03-22_17-39-26_2_frontFar,1280,964,motorcycle,836,523,916,585,JPEGImages/all/BLR-2018-03-22_17-39-26_2_front...
4,0006780.jpg,BLR-2018-03-22_17-39-26_2_frontFar,1280,964,rider,837,518,859,550,JPEGImages/all/BLR-2018-03-22_17-39-26_2_front...


In [21]:
len(train.filename.unique())

4397

In [22]:
len(train.filly_qualified_path.unique())

21458

`There are many images of same name in different directory. So while storing images in different directory i will rename the images and make a list for further reference.`

In [23]:
os.mkdir('train')
for i in train['class'].unique():
    os.mkdir('train/'+i)

In [24]:
#copy all files to a new directory as train
new_file_names = []
for i, j in train.iterrows():
    dest = j['filly_qualified_path'].split('/')[-2]+j['filename']
    new_file_names.append(dest)
    copy(j['filly_qualified_path'],'train/'+dest)

In [25]:
train['new_file_name'] = new_file_names

### repeat for validation

In [26]:
validation = pd.read_csv('validation.csv')
validation.drop(columns=['Unnamed: 0'],inplace=True)
print(validation.shape)
validation.head()

(110440, 10)


,filename,folder,width,height,class,xmin,ymin,xmax,ymax,filly_qualified_path
0,0006780.jpg,BLR-2018-04-16_16-14-27_frontFar,1280,964,truck,401,392,619,488,JPEGImages/all/BLR-2018-04-16_16-14-27_frontFa...
1,0006780.jpg,BLR-2018-04-16_16-14-27_frontFar,1280,964,truck,638,381,835,497,JPEGImages/all/BLR-2018-04-16_16-14-27_frontFa...
2,0006780.jpg,BLR-2018-04-16_16-14-27_frontFar,1280,964,truck,733,379,928,493,JPEGImages/all/BLR-2018-04-16_16-14-27_frontFa...
3,0006780.jpg,BLR-2018-04-16_16-14-27_frontFar,1280,964,person,1093,486,1172,707,JPEGImages/all/BLR-2018-04-16_16-14-27_frontFa...
4,0006780.jpg,BLR-2018-04-16_16-14-27_frontFar,1280,964,motorcycle,1101,426,1144,463,JPEGImages/all/BLR-2018-04-16_16-14-27_frontFa...


In [27]:
os.mkdir('validation')
for i in validation['class'].unique():
    os.mkdir('validation/'+i)

In [28]:
#copy all files to a new directory as validation
new_file_names = []
for i, j in validation.iterrows():
    dest = j['filly_qualified_path'].split('/')[-2]+j['filename']
    new_file_names.append(dest)
    copy(j['filly_qualified_path'],'validation/'+dest)

In [29]:
validation['new_file_name'] = new_file_names

### As we have new unique filenames. Lets create tain_input and validation_input files.
These are the files which will be directly passed to Faster RCNN model.

In [31]:
train['path'] = "train/"+train['new_file_name']

In [32]:
validation['path'] = "validation/"+validation['new_file_name']

In [33]:
outF = open("train_input.txt", "w")
for i, j in train.iterrows():
    #print(str(j['path'])+','+str(j['xmin'])+','+str(j['ymin'])+','+str(j['xmax'])+','+str(j['ymax'])+','+str(j['class']))
    outF.write(str(j['path'])+','+str(j['xmin'])+','+str(j['ymin'])+','+str(j['xmax'])+','+str(j['ymax'])+','+str(j['class']))
    outF.write("\n")
outF.close()

In [34]:
outF1 = open("validation_input.txt", "w")
for i, j in validation.iterrows():
    #print(str(j['path'])+','+str(j['xmin'])+','+str(j['ymin'])+','+str(j['xmax'])+','+str(j['ymax'])+','+str(j['class']))
    outF1.write(str(j['path'])+','+str(j['xmin'])+','+str(j['ymin'])+','+str(j['xmax'])+','+str(j['ymax'])+','+str(j['class']))
    outF1.write("\n")
outF1.close()

### Two sample records from train_input.txt(input for Fasster RCNN model)
<br>train/BLR-2018-03-22_17-39-26_2_frontFar0006780.jpg,586,494,638,551,truck
<br>train/BLR-2018-03-22_17-39-26_2_frontFar0006780.jpg,470,483,595,613,vehicle


### Two sample records from validation_input.txt
<br>validation/BLR-2018-04-16_16-14-27_frontFar0006780.jpg,401,392,619,488,truck
<br>validation/BLR-2018-04-16_16-14-27_frontFar0006780.jpg,638,381,835,497,truck

`We will start with training now in next notebook`